### **Fazendo Importações**

In [ ]:
import time

import pandas as pd
from pandas import concat
from pandas import read_csv
from pandas import DataFrame

from xgboost import XGBRegressor

# biblioteca para gráficos mais simples
from matplotlib import pyplot
pyplot.style.use("ggplot")

# essas bibliotecas permitem criar gráficos interativos
import plotly.offline as py
import plotly.graph_objs as go

from math import sqrt
import numpy as np
from numpy import asarray

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

### **Funções**

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # sequência de input (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # sequência de previsão (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    # exclui valores nulos
    if dropnan:
        agg.dropna(inplace=True)
    return agg



# função para calcular o MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



#função que retornar as colunas a serem excluidas dados os seguintes parâmetros
def drop_columns(n_features,n_in,n_out,index_to_drop):
    total_inputs = (n_features * n_in) - 1
    total_columns = (n_features * n_in) + (n_features * n_out) -1
    print(total_columns)
    const = index_to_drop + 1
    inputs_to_drop = [index_to_drop]
    
    while index_to_drop < total_inputs:
        index_to_drop += const
        inputs_to_drop.append(index_to_drop)
    
    outputs_index = total_inputs+1
    outputs_to_drop = []
    output_to_mantain = index_to_drop + const
    print(output_to_mantain)
    while index_to_drop <= total_columns:
        if index_to_drop < outputs_index:
            index_to_drop += 1
            continue
        if index_to_drop == output_to_mantain:
            output_to_mantain += const
            index_to_drop += 1
            continue
            
        outputs_to_drop.append(index_to_drop)
        index_to_drop += 1
        
    return inputs_to_drop + outputs_to_drop

### **definindo variaveis e lendo csv**

In [ ]:
# define principais variaveis

data_for_test = 1000
n_features = 4
n_in = 7 # tempo atras
n_out = 1 # prever a frente 
index_to_drop = 3 # index do que a gente quer prever no dataset no nosso caso OHLC 0,1,2,3 o close tem o index 3

# Lê o dataset na pasta datasets
data = pd.read_csv('./BTCUSDT(2017-2022)1h.csv', header = None)
# dropa colunas desnecessárias e a primeira linha que contém o cabeçalho
dataset = data.drop(data.columns[[0,5]], axis=1)

dataset = dataset.drop([0])

# transforma o dataset em um modelo supervisionado
dataset = series_to_supervised(dataset,n_in, n_out)
print(dataset)
# Nécessário retirar o valor que pretende ser previsto da sequência de input 
# e os valores que não queremos prever da sequência de output
# lembre-se que dependendo da situação esses valores variam
# comente a linha a baixo e veja o "dataset" completo para saber o que fica e o que sai
dataset.drop(dataset.columns[drop_columns(n_features,n_in,n_out,index_to_drop)], axis=1, inplace=True)

# operações básicas para separar os inputs dos outputs, x e y respectivamente
# Depois há o processo de divisão da sequência de treino x_train e y_train da sequência de teste x_test e y_test

x = dataset.values[:,:-n_out]
y = dataset.values[:,-n_out:]

# O 1000 pode ser substituido por outro valor ele é a quantidade de dados para teste
x_train = x[:-data_for_test,:]
x_test = x[-data_for_test:,:]
y_train = y[:-data_for_test]
y_test = y[-data_for_test:]

      var1(t-7) var2(t-7) var3(t-7) var4(t-7) var1(t-6) var2(t-6) var3(t-6)  \
8        9837.0   9900.03    9600.0    9685.0   9685.02    9720.0    9550.0   
9       9685.02    9720.0    9550.0    9635.0    9634.0   9720.03    9510.0   
10       9634.0   9720.03    9510.0    9719.0    9702.0   9719.98    9600.0   
11       9702.0   9719.98    9600.0   9698.23   9666.28   9924.99   9617.19   
12      9666.28   9924.99   9617.19    9840.0   9832.55   9840.99    9531.0   
...         ...       ...       ...       ...       ...       ...       ...   
35902  41746.98  41894.01  41469.48   41470.0   41470.0  41743.26  41469.99   
35903   41470.0  41743.26  41469.99  41535.93  41535.93  41681.89  41200.02   
35904  41535.93  41681.89  41200.02  41621.11  41621.11  42146.25  41556.49   
35905  41621.11  42146.25  41556.49  42143.04  42143.05   42200.0  41880.01   
35906  42143.05   42200.0  41880.01  42094.59  42094.48   42279.0  41861.94   

      var4(t-6) var1(t-5) var2(t-5)  ... var3(t-2) 

### **Treinando modelo Random Forest**

In [ ]:
y_t = [x[0] for x in y_train]

In [ ]:
trainedModels = []

parameter1 = [10,15]
parameter2 = [20,10]
parameter3 = [10,15]
c = 0
start = time.time()
for p1 in parameter1:
    for p2 in parameter2:
        for p3 in parameter3:
            print('#################')
            model = RandomForestRegressor(max_depth = p1, min_samples_split = p2, min_samples_leaf = p3, verbose =2)
            model.fit(x_train,y_t)
            yhat = model.predict(x_test)
            error = mean_absolute_error(y_test, yhat)
            mse = mean_squared_error(y_test, yhat)
            rmse = sqrt(mse)
            
            trainedModels.append({'model': model, 'MAE': error ,'MSE':mse, 'RMSE':rmse, })
            print('modelo ',c,' treinado erro:',error, mse, rmse)
            c += 1
finish = int(time.time() - start)
print("Os testes levaram: ",finish," segundos")
test = trainedModels
trainedModels = sorted(trainedModels, key = lambda item: item['MAE'])

print('=====Modelos Treinados=====')
print(trainedModels)
print('===========================')

print('Melhor resultado:')
print(trainedModels[0])

#################
building tree 1 of 100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   43.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


modelo  0  treinado erro: 276.51646797498023 150788.87568539358 388.31543323101846
#################
building tree 1 of 100


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   49.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


modelo  6  treinado erro: 275.40568656331567 151020.71834962134 388.61384220022495
#################
building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   47.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
# Plotando o esperado vs o previsto

a = [x for x in yhat]
b = [ x[0] for x in y_test]
actual_chart = go.Scatter(y=b, name= 'Preço real')
predict_chart = go.Scatter(y=a, name= 'Preço previsto')
layout = go.Layout(
          margin = {'t':0,'r':0,'b':0,'l':20},
    title='Preços por ano de construção',
                   yaxis={'title':'Preço do bitcoin em doláres'},
                   xaxis = { 
                    'tickformat': '%d/%m/%Y',
                    'title': {
                      'text': "Data (dia/mês/ano)",
                    }},)
fig = go.Figure(layout = layout, data = [predict_chart,actual_chart])
fig.update_layout(title_text='', title_x=0.5, title_y = 0.86, legend=dict(
    yanchor="bottom",
    y=0.03,
    xanchor="right",
    x=0.99
))
py.iplot(fig)

### **treinando modelo Decisiontree regressor**

In [ ]:
start = time.time()
trainedModels = []

parameter1 = [10,15]
parameter2 = [20,10]
parameter3 = [10,15] 
c = 0
start = time.time()
for p1 in parameter1:
    for p2 in parameter2:
        for p3 in parameter3:
            dt = DecisionTreeRegressor(max_depth=p1, min_samples_leaf=p2, min_samples_split=p3)
            dt.fit(x_train,y_train)
            error = mean_absolute_error(y_test, yhat)
            mse = mean_squared_error(y_test, yhat)
            rmse = sqrt(mse)
            trainedModels.append({'model': dt, 'MAE': error ,'MSE':mse, 'RMSE':rmse, })
            print('modelo ',c,' treinado erro:','MAE:',error, "MSE: ",mse,"RMSE: ",rmse)
        
            c += 1

finish = int(time.time() - start)
print("O treino levou: ",finish," segundos")
yhat = dt.predict(x_test)
test = trainedModels
trainedModels = sorted(trainedModels, key = lambda item: item['MSE'])


print('=====Modelos Treinados=====')
print(trainedModels)
print('===========================')

print('Melhor resultado:')
print(trainedModels[0])


modelo  0  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  1  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  2  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  3  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  4  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  5  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  6  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
modelo  7  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
O treino levou:  7  segundos
=====Modelos Treinados=====
[{'model': DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=10), 'MAE': 6455.6479259375, 'MSE': 42062956.05307078, 'RMSE': 6485.596044549088}, {'mode

In [ ]:
# Plotando o esperado vs o previsto

a = [x for x in yhat]
b = [ x[0] for x in y_test]
actual_chart = go.Scatter(y=b, name= 'Preço real')
predict_chart = go.Scatter(y=a, name= 'Preço previsto')
layout = go.Layout(
          margin = {'t':0,'r':0,'b':0,'l':20},
    title='Preços por ano de construção',
                   yaxis={'title':'Preço do bitcoin em doláres'},
                   xaxis = { 
                    'tickformat': '%d/%m/%Y',
                    'title': {
                      'text': "Data (dia/mês/ano)",
                    }},)
fig = go.Figure(layout = layout, data = [predict_chart,actual_chart])
fig.update_layout(title_text='', title_x=0.5, title_y = 0.86, legend=dict(
    yanchor="bottom",
    y=0.03,
    xanchor="right",
    x=0.99
))
py.iplot(fig)

### ***Treinando modelo Linear regressor***

In [ ]:
start = time.time()
lr = LinearRegression()
lr.fit(x_train,y_train)

finish = int(time.time() - start)
print("O treino levou: ",finish," segundos")

lrYhat = lr.predict(x_test)
mse = mean_squared_error(y_test, lrYhat)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, lrYhat)


print('RMSE: %.6f' % rmse)
print('MSE: %.6f' % mse)
print('MAE: %.6f' % mae)





O treino levou:  0  segundos
RMSE: 370.332741
MSE: 137146.339242
MAE: 256.111802


In [ ]:
# Plotando o esperado vs o previsto


actual_chart = go.Scatter(y=b, name= 'Preço real')
predict_chart = go.Scatter(y=a, name= 'Preço previsto')
layout = go.Layout(
          margin = {'t':0,'r':0,'b':0,'l':20},
    title='Preços por ano de construção',
                   yaxis={'title':'Preço do bitcoin em doláres'},
                   xaxis = { 
                    'tickformat': '%d/%m/%Y',
                    'title': {
                      'text': "Data (dia/mês/ano)",
                    }},)
fig = go.Figure(layout = layout, data = [predict_chart,actual_chart])
fig.update_layout(title_text='', title_x=0.5, title_y = 0.86, legend=dict(
    yanchor="bottom",
    y=0.03,
    xanchor="right",
    x=0.99
))
py.iplot(fig)

### **Treinando Xgbooste regressor**

In [ ]:
trainedModels = []

parameter1 = [15,10,20]
parameter2 = [0.2,0.1,0.02]
parameter3 = [10,15] 
n_estimators = [100]
c = 0
start = time.time()
for p1 in parameter1:
    for p4 in n_estimators:   
        for p2 in parameter2:
            print('#################')
            model2 = XGBRegressor(max_depth = p1, n_estimators = p4,verbosity = 0, learning_rate=p2)
            model2.fit(x_train,y_train)
            yhat = model.predict(x_test)
            error = mean_absolute_error(y_test, yhat)
            mse = mean_squared_error(y_test, yhat)
            rmse = sqrt(mse)
            trainedModels.append({'model': model2, 'MAE': error ,'MSE':mse, '':rmse, })
            print('modelo ',c,' treinado erro:','MAE:',error, "MSE: ",mse,"RMSE: ",rmse)
            c += 1
finish = int(time.time() - start)
print("Os testes levaram: ",finish," segundos")
test = trainedModels
trainedModels = sorted(trainedModels, key = lambda item: item['MAE'])

print('=====Modelos Treinados=====')
print(trainedModels)
print('===========================')

print('Melhor resultado:')
print(trainedModels[0])

#################
modelo  0  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  1  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  2  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  3  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  4  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  5  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  6  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  7  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078 RMSE:  6485.596044549088
#################
modelo  8  treinado erro: MAE: 6455.6479259375 MSE:  42062956.05307078

In [ ]:
# Plotando o esperado vs o previsto

a = [x for x in yhat]
b = [ x[0] for x in y_test]
actual_chart = go.Scatter(y=b, name= 'Preço real')
predict_chart = go.Scatter(y=a, name= 'Preço previsto')
layout = go.Layout(
          margin = {'t':0,'r':0,'b':0,'l':20},
    title='Preços por ano de construção',
                   yaxis={'title':'Preço do bitcoin em doláres'},
                   xaxis = { 
                    'tickformat': '%d/%m/%Y',
                    'title': {
                      'text': "Data (dia/mês/ano)",
                    }},)
fig = go.Figure(layout = layout, data = [predict_chart,actual_chart])
fig.update_layout(title_text='', title_x=0.5, title_y = 0.86, legend=dict(
    yanchor="bottom",
    y=0.03,
    xanchor="right",
    x=0.99
))
py.iplot(fig)